# Workshop: Multi-Agent LLMs and Knowledge Graphs for Monitoring and Sustaining Software Systems

## Create Environment and Install libraries

In [38]:
# pip install python=3.10.18
# pip install neo4j
# pip install -U "autogen-agentchat" "autogen-ext[openai]"
# pip install matplotlib
# pip install networkx

## Docker Installation 

In [37]:
# in bash: 
# docker-compose up -d

## Import libaries

In [39]:
import configparser
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime
from neo4j import GraphDatabase
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console

##  Prepare the model (API key and configuration)

In [3]:
config = configparser.ConfigParser(allow_no_value = True)
config.read('openaiapi.ini')
openai_api_key = config.get('openai', 'OPENAI_API_KEY')

In [28]:
openai_model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=openai_api_key,
    max_tokens = None,
    temperature = None,
    seed = None,
    top_p = None,
    parallel_tool_calls=False  # Disable for Swarms
)


## Setup Neo4J Database 

In [10]:
driver = GraphDatabase.driver("neo4j://localhost:7687", auth=("neo4j", "password"))

# First check for connectivity:
driver.verify_connectivity()

In [25]:
async def query_neo4j(query: str) -> str:
    """Run a Cypher query against Neo4j and return results. Returns String representation."""
    try:
        with driver.session() as session:
            result = session.run(query)
            records = [record.data() for record in result]
        return repr(records)  
    except Exception as e:
        return repr({"error": str(e)})

In [26]:

# Second check for connectivity (run actual query):
await query_neo4j("RETURN 'Checking connectivity with Neo4j!' AS msg")

"[{'msg': 'Checking connectivity with Neo4j!'}]"

## Agent Setup

In [33]:
current_date = datetime.now()
current_date

datetime.datetime(2025, 8, 22, 8, 36, 35, 646181)

In [35]:
graph_agent = AssistantAgent(
    name="graph_operator",
    model_client=openai_model_client,
    tools=[query_neo4j],
    system_message ="""Your name is graph_operator. You generate **Cypher queries** and execute them.
                    You have access to a tool called query_neo4j that you can use to run Cypher queries and optionally retrieve results. 
                    Use the schema below: \
                    Node Types:
                   (1) CodeChange 
                    - Properties: 
                        uid: str
                        timestamp: date
                        change_description: str
                        change_location: List[str]   # files, functions, or classes affected
                   (2) SourceCode 
                      - Properties:
                        uid: str
                        code_description: str
                        lines_of_code: int
                   (3) CommitMessage
                     - Properties:
                        uid: str
                        message_text: str
                        author: str
                        commit_hash: str
                    Relationship Types:
                 (1) (n:CodeChange)-[:]->(m:CodeChange)         # Temporal sequence of changes
                 (2) (n:SourceCode)-[:]->(m:CodeChange)         # Source code elements impacted by the change
                 (3) (n:CommitMessage)-[:]->(m:CodeChange)      # Commit that introduced the change
                    Rules:
                - Only use this schema. Do not invent nodes or relationships.
                - Only one Cypher statement per query is allowed.""",
)

In [34]:
task = f"""
Date: {current_date}.
Commit: a13f9c7 by Alice Smith
Commit Message: "Refactored statistics module to improve clarity and reduce duplication"

Changes:
- Refactoring performed on functions `calculate_sum` and `summarize_report` in `statistics.py`
- Moved helper function `format_output` from `utils/helpers.py` into `statistics.py` for better cohesion
- Reduced lines of code in `summarize_report` from 120 to 80

Context:
- This refactor was done as a follow-up to commit `98df231` (previous bugfix in statistics module).
- Refactor categorized as 'code cleanup' and 'modularity improvement'.
"""

## Start chat:

In [36]:
async def assistant_run_stream() -> None:
    # Option 1: read each message from the stream (as shown in the previous example).
    # async for message in agent.run_stream(task="Find information on AutoGen"):
    #     print(message)

    # Option 2: use Console to print all messages as they appear.
    await Console(
        graph_agent.run_stream(task=task),
        output_stats=True,  # Enable stats printing.
    )


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

---------- TextMessage (user) ----------

Date: 2025-08-22 08:36:35.646181.
Commit: a13f9c7 by Alice Smith
Commit Message: "Refactored statistics module to improve clarity and reduce duplication"

Changes:
- Refactoring performed on functions `calculate_sum` and `summarize_report` in `statistics.py`
- Moved helper function `format_output` from `utils/helpers.py` into `statistics.py` for better cohesion
- Reduced lines of code in `summarize_report` from 120 to 80

Context:
- This refactor was done as a follow-up to commit `98df231` (previous bugfix in statistics module).
- Refactor categorized as 'code cleanup' and 'modularity improvement'.

---------- ToolCallRequestEvent (graph_operator) ----------
[FunctionCall(id='call_vCtrO5k2dTm5XE1SssCG3EaJ', arguments='{"query":"CREATE (c:CodeChange {uid: \'cc1\', timestamp: date(\'2025-08-22\'), change_description: \'Refactored statistics module to improve clarity and reduce duplication\', change_location: [\'statistics.py\', \'utils/helpers.py

## Visualize Graph Content

In [42]:
def visualize_query(query):
    with driver.session() as session:
        result = session.run(query)
        G = nx.DiGraph()
        for record in result:
            start = record["a"]
            end = record["b"]
            rel = record["r"].type
            G.add_edge(start["uid"], end["uid"], label=rel)

    if len(G.nodes) == 0:
        print("No results returned. Check your query or database contents.")
        return

    plt.figure(figsize=(8,6))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_color="lightblue", node_size=2000, font_size=10, arrows=True)
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    plt.show()

In [45]:
visualize_query("MATCH (a)-[r]->(b) RETURN a, r, b LIMIT 20")

No results returned. Check your query or database contents.
